# 2023 CITS4012 Assignment

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

Import necessary packages.

In [1]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import pandas as pd

[nltk_data] Downloading package punkt to /Users/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Read the train and test data and save them as pandas data frames.

In [2]:
train_data = pd.read_csv('WikiQA-train.tsv', sep='\t')
test_data = pd.read_csv('WikiQA-test.tsv', sep='\t')

Extract the unique questions from the train and test data frames

In [3]:
def get_questions(data):
    return data['Question'].unique().tolist()

train_questions = get_questions(train_data)
test_questions = get_questions(test_data)

Extract the answers corresponding to each question from the train and test data frames. If an answer does not exist for a particular question, then an empty string will be added to the answer list.

In [4]:
def get_answers(data, questions):
    answers = []
    for question in questions:
        df = data[data['Question'] == question]
        index = df.loc[df['Label'] == 1]['Sentence'].index.values
        if len(index) == 0:
            answers.append('')
        else:
            answers.append(df.loc[index[0], "Sentence"])
    return answers

train_answers = get_answers(train_data, train_questions)
test_answers = get_answers(test_data, test_questions)

In [23]:
train_answers

['A glacier cave is a cave formed within the ice of a glacier .',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'This tablespoon has a capacity of about 15 mL.',
 'The series also originated much tie-in merchandise, making the Harry Potter brand worth in excess of $15 billion.',
 'A rocket engine, or simply "rocket", is a jet engine that uses only stored propellant mass for forming its high speed propulsive jet .',
 '',
 'Transmission occurs primarily by drinking water or eating food that has been contaminated by the feces (waste product) of an infected person, including one with no apparent symptoms.',
 '',
 '',
 '',
 '',
 'Anne Frank and her sister, Margot , were eventually transferred to the Bergen-Belsen concentration camp , where they died of typhus in March 1945.',
 '',
 'The radial engine is a reciprocating type internal combustion engine configuration in which the cylinders point outward from a central crankshaft like the spokes of a wheel.',
 '',
 'Deepwater drilling is t

Extract the documents corresponding to each question by joining the sentences. Make sure each sentence is seperated by a whitespace.

In [5]:
def get_documents(data, questions):
    documents = []
    for question in questions:
        df = data[data['Question'] == question]
        sentences = df['Sentence'].tolist()
        for i in range(0, len(sentences) - 1):
            sentences[i] = sentences[i] + ' '
        documents.append(''.join(sentences))
    return documents

train_documents = get_documents(train_data, train_questions)
test_documents = get_documents(test_data, test_questions)

Tokenize documents and label tokens as: Start Token of the Answer (S), Inner Token of the Answer (I), End Token of the Answer (E) or None (N). 

In [11]:
def label_documents(documents, answers):
    tagged_documents = []
    for i in range(len(documents)):
        tagged_document = []
        answer = answers[i]
        document = documents[i]
        if answer == '':
            tokens = word_tokenize(document)
            for j in range(len(tokens)):
                tagged_document.append('N')
        else:
            parts = document.partition(answer)
            for j in range(len(parts)):
                tokens = word_tokenize(parts[j])
                if j == 1:
                    tagged_document.append('S')
                    for k in range(len(tokens) - 2):
                        tagged_document.append('I')
                    tagged_document.append('E')
                else:
                    for k in range(len(tokens)):
                        tagged_document.append('N')
            tagged_documents.append(tagged_document)
    return tagged_documents

train_labeled_documents = label_documents(train_documents, train_answers)
test_labeled_documents = label_documents(test_documents, test_answers)

In [14]:
zip(train_labeled_documents[0], train_documents[0])

In [16]:
len(train_labeled_documents[0])

62

In [25]:
train_documents[1]

"In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path. It can be uniform, with constant angular rate of rotation (and constant speed), or non-uniform with a changing rate of rotation. The rotation around a fixed axis of a three-dimensional body involves circular motion of its parts. The equations of motion describe the movement of the center of mass of a body. Examples of circular motion include: an artificial satellite orbiting the Earth at constant height, a stone which is tied to a rope and is being swung in circles, a car turning through a curve in a race track , an electron moving perpendicular to a uniform magnetic field , and a gear turning inside a mechanism. Since the object's velocity vector is constantly changing direction, the moving object is undergoing acceleration by a centripetal force in the direction of the center of rotation. Without this acceleration, the object would move in a straight line, a

# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

### 3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



### 3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

### 3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 